In [1]:
path_snap_06 = "/mnt/data1/sandeep/New_Gadget2_run/200Mpc_256/snapdir_seed_1690811/snap_06"

In [8]:
import matplotlib.pyplot as plt

## Reading the snapshot

In [9]:

import numpy as np
import sys
import os
import astropy.units as u
import pandas as pd


class Gadget2_snapshot_hdr:

    def __init__(self, filename, DataFrame=False):

        if os.path.exists(filename):
            inp_file = filename
        else:
            print "file not found:"
            sys.exit()
        self.filename = filename   # keepig filename as self variable 
        self.Data_frame=DataFrame
        f = open(inp_file,'rb')    # rb is for read binary
        blocksize = np.fromfile(f,dtype=np.int32,count=1)
        if blocksize[0]!=256:
            raise ValueError("incorrect file format encountered when reading header of")

	#==== Gadget2 struct_io_header =====
	# number of particles of each type in this file 
        self.npart = np.fromfile(f,dtype=np.int32,count=6)

	# mass of particles of each type. If 0, then the masses are explicitly
	# stored in the mass-block of the snapshot file, otherwise they are omitted 
        self.massarr   = np.fromfile(f,dtype=np.float64,count=6)

	# time of snapshot file 
        self.time      = (np.fromfile(f,dtype=np.float64,count=1))[0]

	# redshift of snapshot file 
        self.redshift  = (np.fromfile(f,dtype=np.float64,count=1))[0]

	# flags whether the simulation was including star formation 
        self.flag_sfr  = (np.fromfile(f,dtype=np.int32,count=1))[0]

	# flags whether feedback was included (obsolete) 
        self.flag_feedback = (np.fromfile(f,dtype=np.int32,count=1))[0]

	# total number of particles of each type in this snapshot. This can be
	# different from npart if one is dealing with a multi-file snapshot. 
        self.npartTotal    = np.fromfile(f,dtype=np.uint32,count=6)

	# flags whether cooling was included  */
        self.flag_cooling  = (np.fromfile(f,dtype=np.int32,count=1))[0]

	# number of files in multi-file snapshot 
        self.num_files     = (np.fromfile(f,dtype=np.int32,count=1))[0]

	# box-size of simulation in case periodic boundaries were used 
        self.BoxSize       = (np.fromfile(f,dtype=np.float64,count=1))[0]

	# matter density in units of critical density 
        self.Omega0        = (np.fromfile(f,dtype=np.float64,count=1))[0]

	# cosmological constant parameter 
        self.OmegaLambda   = (np.fromfile(f,dtype=np.float64,count=1))[0]

	# Hubble parameter in units of 100 km/sec/Mpc 
        self.HubbleParam   = (np.fromfile(f,dtype=np.float64,count=1))[0]

	# flags whether the file contains formation times of star particles 
        self.flag_stellarage = (np.fromfile(f,dtype=np.float32,count=1))[0]

	# flags whether the file contains metallicity values for gas and star particles 
        self.flag_metals =  (np.fromfile(f,dtype=np.float32,count=1))[0]

     	# High word of the total number of particles of each type 
        self.npartTotalHighWord = np.fromfile(f,dtype=np.uint32,count=6)

        # flags that IC-file contains entropy instead of u 
        self.flag_entropy_instead_u = (np.fromfile(f,dtype=np.float32,count=1))[0]

        # fills to 256 Bytes 
        self.unused = np.fromfile(f,dtype=np.byte, count=60)
        
        blk_check = np.fromfile(f,dtype=np.int32,count=1)
        if blocksize[0]!=blk_check[0]:
            raise ValueError("I/O:ERRORBLOCKS NOT MATCHING")
        del blocksize, blk_check 
        f.close()
        
   #---------------------------------------------------------------------------------

    def read_gadget_header(self):
        
	"""
	Simply prints out the header of the file
	""" 

        print 'npar=',self.npart
        print 'nall=',self.npartTotal
        print 'a=',self.time
        print 'z=',self.redshift
        print 'masses=',self.massarr*1e10,'Msun/h'
        print 'boxsize=',self.BoxSize,'kpc/h'
        print 'filenum=',self.num_files
        print 'cooling=',self.flag_cooling
        print 'Omega_m,Omega_l=',self.Omega0,self.OmegaLambda
        print 'h=',self.HubbleParam,'\n'
        print 'H0=', self.HubbleParam*100.* u.km*u.s**(-1)*u.Mpc**(-1)

        rhocrit=2.77536627e11 #h**2 M_sun/Mpc**3
        rhocrit=rhocrit/1e9 #h**2M_sun/kpc**3
        
        Omega_CDM=self.npartTotal[1]*self.massarr[1]*1e10/(self.BoxSize**3*rhocrit)
        print 'DM mass=%.5e  Omega_DM = %.5f'\
          %(self.massarr[1]*1e10, Omega_CDM)

    #------------- Read Particle Positions in comoving units kpc h^-1 ------------------
    
    def read_Pos(self, Double=False):
        
	"""
	Param
	Double: If True then positions are written in 
		double else they are written in 
	
	returns
	If Data_frame is flase then returns the numpy 
	array of shape (3, Npart) else Panda's data frame
	of the same shape.  
	"""

        with open(self.filename,'rb') as f:   # rb is for read binary
                offset = 4+256+4

                f.seek(offset, os.SEEK_CUR) # https://www.geeksforgeeks.org/python-seek-function/(???????)
                blocksize = np.fromfile(f,dtype=np.int32,count=1) 
                #https://numpy.org/doc/stable/reference/generated/numpy.fromfile.html
                
                if not Double:
                        dt = np.dtype((np.float32,3))  # Positions are in float not double
                else:     
                        dt = np.dtype((np.float64,3))  # Positions are in float not double

                Pos = np.fromfile(f,dtype=dt,count=self.npart[1]) # https://numpy.org/doc/stable/reference/arrays.dtypes.html
                blk_check = np.fromfile(f,dtype=np.int32,count=1)

                if blocksize[0]!=blk_check[0]:
                        raise ValueError("I/O:ERRORBLOCKS NOT MATCHING")
                
                del blk_check, blocksize
 
        
        if self.Data_frame == True:
            df = pd.DataFrame(Pos)
            return df
        else:
            return Pos

    #-------- Read Particle velocities in Gadget internal units as km s^-1 ---------

    def read_Vel(self):
    
	"""
	returns
	If Data_frame is flase then returns the numpy 
	array of shape (3, Npart) else Panda's data frame
	of the same shape.  
	"""
       
        f = open(self.filename,'rb')  # rb is for read binary
        
        # Calculate the offset from the beginning of the 
	# file: 4 bytes (endianness) + 256 bytes (header) + 8 bytes (void)

        offset = 4+256+8
        
        #Skip all the particle Position (four bytes per position * three coordinates* number of particles)
        offset += 4 * 3 * self.npart[1]
        f.seek(offset+4, os.SEEK_CUR)
        
        blocksize = np.fromfile(f,dtype=np.int32,count=1)
        dt = np.dtype((np.float32,3))  # velocities are in float not double
        Vel = np.fromfile(f,dtype=dt,count=self.npart[1])

        blk_check = np.fromfile(f,dtype=np.int32,count=1)
        
        if blocksize[0]!=blk_check[0]:
            raise ValueError("I/O:ERRORBLOCKS NOT MATCHING")
                    
        del blk_check, blocksize
        f.close()    
        
        if self.Data_frame == True:
            df = pd.DataFrame(Vel)
            return df
        else:
            return Vel
        

    #----------------- Read Particle ID --------------------------

    def read_ID(self):
    
        f = open(self.filename,'rb')    # rb is for read binary
        
        # Calculate the offset from the beginning of the 
	# file: 4 bytes (endianness) + 256 bytes (header) + 8 bytes (void)
        
        offset = 4+256+8
        #Skip all the particle Position
        offset += 4 * 3 * self.npart[1]
        offset+=8
        #Skip all the particle velocities
        offset += 4 * 3 * self.npart[1]

        f.seek(offset+4, os.SEEK_CUR)
        
        blocksize = np.fromfile(f,dtype=np.int32,count=1)

        Part_ID = np.fromfile(f,dtype=np.uint32, count=self.npart[1])
        blk_check = np.fromfile(f,dtype=np.int32,count=1)
        
        if blocksize[0]!=blk_check[0]:
            raise ValueError("I/O:ERRORBLOCKS NOT MATCHING")
            
        del blk_check, blocksize
        f.close()    

        return Part_ID

## Reading the Header

In [10]:
import argparse
import numpy as np
import sys
sys.path.insert(1, '/mnt/home1/sandeep/workspace/Simulations_workspace/python_routines/Gadget2_routines/')




snapID = np.int32(22)
nfiles = np.int32(32)
dir_name= ("/mnt/data1/sandeep/Kalinga_run_Analysis/512Mpc_1024/kalinga_snapdir_seed_1690811/")

#/mnt/data1/sandeep/New_Gadget2_run/200Mpc_256/snapdir_seed_1690811/snap_06
print dir_name 
if nfiles<=1:
    inp_file = dir_name + "snapshot_%03d"%(snapID)
else:
    inp_file = dir_name + "snapshot_%03d.%d"%(snapID, 0)
    print(inp_file)

snapshot = Gadget2_snapshot_hdr(inp_file, DataFrame=True)
snapshot_header =  snapshot.read_gadget_header()





/mnt/data1/sandeep/Kalinga_run_Analysis/512Mpc_1024/kalinga_snapdir_seed_1690811/
/mnt/data1/sandeep/Kalinga_run_Analysis/512Mpc_1024/kalinga_snapdir_seed_1690811/snapshot_022.0
npar= [       0 33979285        0        0        0        0]
nall= [         0 1073741824          0          0          0          0]
a= 1.0
z= 4.4408920985e-16
masses= [  0.00000000e+00   1.06549701e+10   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00] Msun/h
boxsize= 512000.0 kpc/h
filenum= 32
cooling= 0
Omega_m,Omega_l= 0.307115 0.692885
h= 0.6777 

H0= 67.77 km / (Mpc s)
DM mass=1.06550e+10  Omega_DM = 0.30713


## Data

In [12]:
import numpy as np
import os
import argparse
from astropy.io import ascii

    

scale_fac = 1./(1.+snapshot.redshift)
unit_kpc_Mpc = np.float32(0.001)
vel_conv_factor = np.float32(1./np.sqrt(scale_fac))

posx = np.zeros(1, dtype=np.float64)
posy = np.zeros(1, dtype=np.float64)
posz = np.zeros(1, dtype=np.float64)

velx = np.zeros(1, dtype=np.float64)
vely = np.zeros(1, dtype=np.float64)
velz = np.zeros(1, dtype=np.float64)

for i in xrange(nfiles):

    if nfiles<=1:
        inp_file = dir_name + "snapshot_%03d"%(snapID)
    else:
        inp_file = dir_name + "snapshot_%03d.%d"%(snapID, i)
    print(inp_file)

    snapshot = Gadget2_snapshot_hdr(inp_file, DataFrame=False)
    Pos  = snapshot.read_Pos() # is u.kpc*snapshot.HubbleParam**(-1)

    Pos[:,0] = np.float64(Pos[:,0]*unit_kpc_Mpc)
    Pos[:,1] = np.float64(Pos[:,1]*unit_kpc_Mpc)
    Pos[:,2] = np.float64(Pos[:,2]*unit_kpc_Mpc)

    posx = np.concatenate((posx, Pos[:,0]))
    posy = np.concatenate((posy, Pos[:,1]))
    posz = np.concatenate((posz, Pos[:,2]))


    Vel  = snapshot.read_Vel() # to convert the Gadget velocity to comoving velocity
    Vel[:,0] *= np.float64(vel_conv_factor)
    Vel[:,1] *= np.float64(vel_conv_factor)
    Vel[:,2] *= np.float64(vel_conv_factor)

    velx = np.concatenate((velx, Vel[:,0]))        
    vely = np.concatenate((vely, Vel[:,1]))        
    velz = np.concatenate((velz, Vel[:,2]))        
    del Pos, Vel 




/mnt/data1/sandeep/Kalinga_run_Analysis/512Mpc_1024/kalinga_snapdir_seed_1690811/snapshot_022.0
/mnt/data1/sandeep/Kalinga_run_Analysis/512Mpc_1024/kalinga_snapdir_seed_1690811/snapshot_022.1
/mnt/data1/sandeep/Kalinga_run_Analysis/512Mpc_1024/kalinga_snapdir_seed_1690811/snapshot_022.2
/mnt/data1/sandeep/Kalinga_run_Analysis/512Mpc_1024/kalinga_snapdir_seed_1690811/snapshot_022.3
/mnt/data1/sandeep/Kalinga_run_Analysis/512Mpc_1024/kalinga_snapdir_seed_1690811/snapshot_022.4
/mnt/data1/sandeep/Kalinga_run_Analysis/512Mpc_1024/kalinga_snapdir_seed_1690811/snapshot_022.5
/mnt/data1/sandeep/Kalinga_run_Analysis/512Mpc_1024/kalinga_snapdir_seed_1690811/snapshot_022.6
/mnt/data1/sandeep/Kalinga_run_Analysis/512Mpc_1024/kalinga_snapdir_seed_1690811/snapshot_022.7
/mnt/data1/sandeep/Kalinga_run_Analysis/512Mpc_1024/kalinga_snapdir_seed_1690811/snapshot_022.8
/mnt/data1/sandeep/Kalinga_run_Analysis/512Mpc_1024/kalinga_snapdir_seed_1690811/snapshot_022.9
/mnt/data1/sandeep/Kalinga_run_Analysis/

In [14]:
##########################functions to find ngp and cic####################################

def ngp(ngrid, slice_width, nbox, posi, offset):
    posi[2] -= offset
    new_pos = np.swapaxes(posi,0,1)
    boxfac = np.float32(ngrid/nbox)
    density = np.zeros((ngrid+1, ngrid+1, np.int32(boxfac*slice_width)+1)) #defining box of size ngrid
    npart = len(new_pos) 
    new_pos = boxfac*new_pos
    ints = np.floor(new_pos).astype(np.int32)  
    print(ints.min(axis =0))
    for i in xrange(len(ints)):
        density[ints[i][0], ints[i][1], ints[i][2]] += 1
                
    return density


def cic(ngrid,slice_width, nbox, posi, offset):
    posi[2] -= offset
    new_pos = np.swapaxes(posi,0,1)
    boxfac = np.float32(ngrid/nbox)
    density = np.zeros((ngrid+1, ngrid+1, np.int32(boxfac*slice_width)+1), dtype=np.float64) #defining box of size ngrid
    
    npart = len(new_pos) 
    new_pos = boxfac*new_pos
    
    frac = np.modf(new_pos)[0]
    ints = np.floor(new_pos).astype(np.int32)
                                                                                                                                                                                                                                                                                                                                                   
    for i in xrange(-1,len(ints)-1):
        for j in range(2):
            if j == 1 : frac[i][0] = 1 - frac[i][0]
            for k in range(2):
                if k == 1 : frac[i][1] = 1 - frac[i][1]
                for l in range(2):
                    if l == 1 : frac[i][2] = 1 - frac[i][2]
                    density[ints[i+j][0], ints[i+k][1], ints[i+l][2]] += (frac[i][0])*(frac[i][1])*(frac[i][2])
                    
            
            
        
#         density[ints[i+1][0], ints[i][1], ints[i][2]] += (frac[i][0])*(1 - frac[i][1])*(1 - frac[i][2])
#         density[ints[i][0], ints[i+1][1], ints[i][2]] += (1 - frac[i][0])*(frac[i][1])*(1 - frac[i][2])
#         density[ints[i][0], ints[i][1], ints[i+1][2]] += (1 - frac[i][0])*(1 - frac[i][1])*(frac[i][2])
#         density[ints[i+1][0], ints[i+1][1], ints[i][2]] += (frac[i][0])*(frac[i][1])*(1 - frac[i][2])
#         density[ints[i][0], ints[i+1][1], ints[i+1][2]] += (1 - frac[i][0])*(frac[i][1])*(frac[i][2])
#         density[ints[i+1][0], ints[i][1], ints[i+1][2]] += (frac[i][0])*(1 - frac[i][1])*(frac[i][2])
#         density[ints[i][0], ints[i][1], ints[i][2]] += (1 - frac[i][0])*(1 - frac[i][1])*(1 - frac[i][2])
#         density[ints[i+1][0], ints[i+1][1], ints[i+1][2]] += (frac[i][0])*(frac[i][1])*(frac[i][2])
        
        

    return density

###########################################################################################################

## Slicing

In [15]:
Slice_width = np.float64(10)
p = np.float64(0)
index_z = (posz>=p + 0.0)*(posz<=p+Slice_width)
positions = np.array([(posx[index_z])[1:], (posy[index_z])[1:], (posz[index_z])[1:]])

In [ ]:
Slice_width = np.float64(10)
for p in xrange(0,5,10):
    p = np.float64(p)
    index_z = (posz>=p + 0.0)*(posz<=p+Slice_width)
    #print len(posx[index_z])

    # my_table = [(posx[index_z])[1:], (posy[index_z])[1:], (posz[index_z])[1:],
    #             (velx[index_z])[1:], (vely[index_z])[1:], (velz[index_z])[1:]
    #            ]
    positions = np.array([(posx[index_z])[1:], (posy[index_z])[1:], (posz[index_z])[1:]])
    print(positions)

    positions.min()

    import numpy as np
    import matplotlib.pyplot as plt
    from multiprocessing import Process
    import matplotlib.cm as cm
    from matplotlib.colors import LogNorm
    import copy
    #from de_funcs import ngp, cic


    pos = copy.deepcopy(positions)  
    pos1 = copy.deepcopy(positions) 
    print np.swapaxes(pos,0,1).min(axis = 0)





    ngrid = np.int32(1024)
    nbox = np.float64(512)
    lengrid = np.float64(nbox/ngrid)
    slice_width = np.float64(10)

    numgrid = lengrid*slice_width
    print(numgrid)
    npart = np.int32(len(pos[0]))

    #finding ngp and cic 3-D arrays

    den = ngp(ngrid,slice_width,nbox,pos,p)
    denc = cic(ngrid,slice_width,nbox,pos1,p)




    #density check for cic
    if int(denc.sum()) == npart : print('CIC is consistent with the mass density distribution')
    print(denc.sum())
    print(len(pos[0]))




    #density check for ngp
    if int(den.sum()) == npart: print('NGP is consistent with the mass density distribution')
    print(den.sum())
    print(len(pos[0]))




    #slicing the density
    grid_num = 0
    print(den.shape)
    den_2d = den[ :,:,grid_num]
    denc_2d = denc[ :,:,grid_num]
    for i in range(1,int(numgrid)+1):
        den_2d += den[ :,:,grid_num + i]
        denc_2d += denc[ :,:,grid_num + i]




  

    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    from matplotlib.colors import LogNorm
    import numpy as np


    min_den = den_2d.min()
    max_den = den_2d.max()

    min_denc = denc_2d.min()
    max_denc = denc_2d.max()

    print(min_den, max_den, min_denc, max_denc)

    fig = plt.figure(figsize = (15,15))
    ax = fig.add_subplot(1,1,1)
    im = ax.imshow(denc_2d, 
            cmap=cm.afmhot, norm=LogNorm(vmin =0.8, vmax= 1000))
    # ax1 = fig.add_subplot(1,2,2)
    # im1 = ax1.imshow(denc_2d, 
    #         cmap=cm.afmhot, norm=LogNorm(vmin =0.8, vmax= 1000))
    ax.savefig(str(p)+'cic_10Mpc_'+str(ngrid)+'g.eps', format='eps')

    plt.show()

In [ ]:
#positions1 = np.array([(posx[:])[1:], (posy[:])[1:], (posz[:])[1:]])

np.save('pos_512.npy', positions, fix_imports = True)

## Gaussian kde kernel

In [ ]:
import copy
print positions.shape
X = copy.deepcopy(positions[0])
Y = copy.deepcopy(positions[1])
#print(X)

from scipy.stats import gaussian_kde                                                                                                                                                                                                                                            

fig = plt.figure(figsize=(8, 8))
xy = np.vstack([X,Y])
print xy.shape
z = gaussian_kde(xy)
print type(gaussian_kde(xy))



plt.scatter(X, Y, c=z(xy), s=0.001)
plt.show()

# np.save('pos(w_1).npy', positions,fix_imports = True)

Nx, Ny = 512, 512
X, Y = np.meshgrid(np.arange(Nx), np.arange(Ny))
print(X,Y)


xy=np.vstack([X.flatten(),Y.flatten()])
plt.scatter(X.flatten(), Y.flatten(),c=z(xy), s=10)
plt.show()

(3, 20169011)
(2, 20169011)
<class 'scipy.stats.kde.gaussian_kde'>


## Preliminary code for SPH

In [ ]:
def W( x, y, z, h ):
	"""
    Gausssian Smoothing kernel (3D)
	x     is a vector/matrix of x positions
	y     is a vector/matrix of y positions
	z     is a vector/matrix of z positions
	h     is the smoothing length
	w     is the evaluated smoothing function
	"""
	
	r = np.sqrt(x**2 + y**2 + z**2)
	
	w = (1.0 / (h*np.sqrt(np.pi)))**3 * np.exp( -r**2 / h**2)
	
	return w
	
def getPairwiseSeparations( ri, rj ):
	"""
	Get pairwise desprations between 2 sets of coordinates
	ri    is an M x 3 matrix of positions
	rj    is an N x 3 matrix of positions
	dx, dy, dz   are M x N matrices of separations
	"""
	
	M = ri.shape[0]
	N = rj.shape[0]
	
	# positions ri = (x,y,z)
	rix = ri[:,0].reshape((M,1))
	riy = ri[:,1].reshape((M,1))
	riz = ri[:,2].reshape((M,1))
	
	# other set of points positions rj = (x,y,z)
	rjx = rj[:,0].reshape((N,1))
	rjy = rj[:,1].reshape((N,1))
	rjz = rj[:,2].reshape((N,1))
	
	# matrices that store all pairwise particle separations: r_i - r_j
	dx = rix - rjx.T
	dy = riy - rjy.T
	dz = riz - rjz.T
	
	return dx, dy, dz
	

def getDensity( r, pos, m, h ):
	"""
	Get Density at sampling loctions from SPH particle distribution
	r     is an M x 3 matrix of sampling locations
	pos   is an N x 3 matrix of SPH particle positions
	m     is the particle mass
	h     is the smoothing length
	rho   is M x 1 vector of accelerations
	"""
	
	M = r.shape[0]
	
	dx, dy, dz = getPairwiseSeparations( r, pos );
	
	rho = np.sum( m * W(dx, dy, dz, h), 1 ).reshape((M,1))
	
	return rho

In [ ]:
Nx, Ny ,Nz= 512, 512 ,10
X, Y, Z = np.meshgrid(np.arange(Nx), np.arange(Ny), np.arange(Nz))
h = plt.contourf(getDensity(np.vstack([X,Y,Z]),np.vstack([positions[0],positions[1],positions[2]]),1))